In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(3, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Queue(model, 'Queue2', SchedStrategy.PS)

jobclass = ClosedClass(model, 'Class1', 1, node[0], 0)

node[0].set_service(jobclass, HyperExp.fit_mean_and_scv(1.0, 25.0))
node[1].set_service(jobclass, Exp(1.0))
node[2].set_service(jobclass, Exp(2.0))

model.add_link(node[0], node[0])
model.add_link(node[0], node[1])
model.add_link(node[0], node[2])
model.add_link(node[1], node[0])
model.add_link(node[2], node[0])

node[0].set_routing(jobclass, RoutingStrategy.RROBIN)
node[1].set_prob_routing(jobclass, node[0], 1.0)
node[2].set_prob_routing(jobclass, node[0], 1.0)

In [ ]:
solver = np.array([], dtype=object)
solver = np.append(solver, CTMC(model, keep=True))
solver = np.append(solver, JMT(model, samples=100000, seed=23000))
solver = np.append(solver, SSA(model, verbose=True, samples=10000, seed=23000))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].avg_table()
    print(AvgTable[s])